In [ ]:
%matplotlib inline


神经网络
===============

可以使用`torch.nn`包构建神经网络。

现在您已经了解了`autograd`，`nn`依赖于`autograd`来定义模型并进行微分。`nn.Module`包含层，以及返回`output`的`forward(input)`方法。


例如，查看对数字图像进行分类的网络：

![convnet](https://ptorch.com/uploads/2017072622594219139.jpg
)

它是一个简单的前馈网络。它接受输入，一个接一个地通过几个层输入，然后最终给出输出。



一个典型的神经网络的训练过程如下：

- 定义具有一些可学习参数（或权重）的神经网络
- 迭代输入数据集
- 通过网络处理输入
- 计算损失（输出距离正确多远）
- 将梯度传播回网络参数
- 通常使用简单的更新规则更新网络权重：
  ``weight = weight - learning_rate * gradient``

定义网络
------------------

我们来定义这个网络：



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)




您只需定义`forward`函数，`autograd`会自动为您定义`backward`函数（在其中计算梯度）。您可以在`forward()`函数中使用任何Tensor操作。

模型的可学习参数由``net.parameters()``返回




In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])



让我们尝试随机的32x32输入

注意：此网络（LeNet）的预期输入大小为32x32。要在MNIST数据集上使用此网络，请将数据集中的图像大小调整为32x32。



In [3]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0800, -0.0182,  0.0196, -0.0267, -0.0779, -0.1084,  0.0022,
         -0.1306,  0.1187,  0.0275]])



使用随机梯度将所有参数和反向传播的梯度缓冲区归零：



In [4]:
net.zero_grad()
out.backward(torch.randn(1, 10))

<div class="alert alert-info"><h4>注意</h4><p>``torch.nn`` 支持 mini-batches. 整个``torch.nn``
    package仅支持小批量样本的输入，而不支持单个样本。

    举个例子， ``nn.Conv2d`` 以一个4D Tensor 
    ``nSamples x nChannels x Height x Width``为输入。

    如果您有一个样本，只需使用``input.unsqueeze(0)``添加假批量维度。
</p></div>

在继续之前，让我们回顾一下你到目前为止看到的所有课程。

**回顾：**
  - ``torch.Tensor``  - 一个支持autograd操作例如``backward()``的多维数组*，*也根据张量维持梯度* 。

  -  ``nn.Module`` - 神经网络模块。**方便的封装参数的方法**，使用帮助程序将它们移动到GPU，导出，加载等。

  -  ``nn.Parameter`` - 一种Tensor，当被指定为``Module``的属性时，它会自动注册为Parameter。

  -  ``autograd.Function`` - 实现自动求导操作的**Forward和Backward定义**。每个``Tensor``操作，至少创建一个``Function``节点，它连接到创建``Tensor``和*编码其历史*的函数。


**At this point, we covered:**
  -  定义神经网络
  -  处理输入并向后调用


**Still Left:**
  -  计算损失
  -  更新网络权重

损失函数
-------------
损失函数采用（输出，目标）输入对，并计算估计输出距目标的距离的值。


有几种不同
`损失函数 <http://pytorch.org/docs/nn.html#loss-functions>`_ 在nn包下。

一个简单的损失是：``nn.MSELoss``，它计算输入和目标之间的均方误差。

例如：



In [5]:
output = net(input)
target = torch.randn(10)  # 例如，虚拟目标
target = target.view(1, -1)  # 使其与输出形状相同
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.3080)


现在，如果你在Backward方向上跟踪``loss``，使用它的``.grad_fn``属性，你会看到一个如下所示的计算图：


::

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

So, when we call ``loss.backward()``, the whole graph is differentiated
w.r.t. the loss, and all Tensors in the graph that has ``requires_grad=True``
will have their ``.grad`` Tensor accumulated with the gradient.

因此，当我们调用``loss.backward（）``时，整个图对损失微分，图中所有具``requires_grad = True``的张量将会有梯度累积的``.grad``张量。



为了说明，让我们跟随几步反向传播：




In [ ]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

反向传播
--------
要反向传播误差，我们所要做的就是``lost.backward（）``。您需要清除现有梯度，否则梯度将累积到现有梯度。



现在我们将调用``loss.backward（）``，并查看conv1在反向传播之前和之后的梯度偏差。




In [ ]:
net.zero_grad()     # 将所有参数的梯度缓冲区归零

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

现在，我们已经看到了如何使用损失函数。

**进一步阅读:**

  神经网络包包含形成深度神经网络构建块的各种模块和损失函数。带有文档的完整列表是`here <http://pytorch.org/docs/nn>`_。


**最后要学习的是:**

  - 更新网络权重

更新权重
------------------
实践中使用的最简单的更新规则是随机梯度下降（SGD）：

     ``weight = weight - learning_rate * gradient``

我们可以使用简单的python代码实现它：

.. code:: python

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

但是，当您使用神经网络时，您希望使用各种不同的更新规则，例如SGD，Nesterov-SGD，Adam，RMSProp等。
为了实现这一点，我们构建了一个小包：``torch.optim``，它实现了所有这些方法。非常容易使用：



In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # 将梯度缓冲区归零
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # 更新

注意

  观察如何使用``optimizer.zero_grad()``手动将梯度缓冲区设置为零。这是因为梯度是按照`Backprop`部分中的说明累积的。


